In [1]:
import numpy as np
import pandas as pd

In [2]:
data_path = 'data/iris.data'

### Dataset Description

- Number of features for each data-point: 4
- Number of classes: 3 (although we need to cluster the data points to identify what works the best)

### Read data from file in X, y

In [3]:
iris = pd.read_csv(data_path, sep=',', names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type'])

In [4]:
iris

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
for i, flower_type in enumerate(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']):
    iris.loc[iris['type'] == flower_type, 'type'] = i

In [13]:
X = iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy()

In [15]:
X.shape

(150, 4)

In [16]:
y = iris[['type']].to_numpy()

In [17]:
y.shape

(150, 1)